### Prieskumná analýza

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display, HTML

SEED = 42
np.random.seed(SEED)

DATA_DIR = "./115"

# Načítanie dát

df_obs = pd.read_csv(os.path.join(DATA_DIR, "observation.csv"), sep='\t', encoding='utf-8-sig')
df_pat = pd.read_csv(os.path.join(DATA_DIR, "patient.csv"), sep='\t', encoding='utf-8-sig')
df_sta = pd.read_csv(os.path.join(DATA_DIR, "station.csv"), sep='\t', encoding='utf-8-sig')

In [18]:
# --- A. Analýza štruktúr dát ---

# Funkcia na rýchly prehľad datasetu

def dataset_summary(df, name):
    print("Dataset:",name)
    print("Počet riadkov: ",df.shape[0])
    print("Počet stĺpcov: ",df.shape[1])
    display(df.head())
    display(df.info())


dataset_summary(df_obs, "observation.csv")
dataset_summary(df_pat, "patient.csv")
dataset_summary(df_sta, "station.csv")



Dataset: observation.csv
Počet riadkov:  12169
Počet stĺpcov:  23


,SpO₂,HR,PI,RR,EtCO₂,FiO₂,PRV,BP,Skin Temperature,Motion/Activity index,...,CO,Blood Flow Index,PPG waveform features,Signal Quality Index,Respiratory effort,O₂ extraction ratio,SNR,oximetry,latitude,longitude
0,96.634591,80.009013,8.685891,17.518665,41.509058,45.163316,144.295418,104.803180,34.978370,10.828257,...,4.088809,40.790753,53.666253,62.281085,54.919133,0.300000,38.410597,1.0,25.67927,-80.31727
1,96.640621,74.589540,13.724679,15.588010,40.198284,50.791918,110.819791,103.974975,35.193140,9.818858,...,4.036563,43.207084,58.219436,45.805412,58.818949,0.213967,39.018173,0.0,-23.79750,-48.59278
2,98.146757,85.019981,7.766128,18.747887,41.964458,63.858679,91.591179,106.573202,35.938901,9.767053,...,4.224814,48.594592,56.163501,45.538311,46.333772,0.205624,30.761858,1.0,-5.41667,38.01667
3,98.391300,73.823193,11.114940,15.905309,42.781757,55.433424,59.373133,103.438804,34.843787,12.695630,...,4.030798,51.561544,42.193516,44.665135,27.899934,0.292025,28.550364,1.0,34.56861,105.89333
4,97.391221,78.702018,12.126423,14.923299,38.946213,52.511153,100.147921,102.652685,36.109030,10.813473,...,4.070682,51.560475,45.447019,63.837901,50.321388,0.242503,38.698971,1.0,51.65333,5.28750


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12169 entries, 0 to 12168
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   SpO₂                   12169 non-null  float64
 1   HR                     12169 non-null  float64
 2   PI                     12169 non-null  float64
 3   RR                     12169 non-null  float64
 4   EtCO₂                  12169 non-null  float64
 5   FiO₂                   12169 non-null  float64
 6   PRV                    12169 non-null  float64
 7   BP                     12169 non-null  float64
 8   Skin Temperature       12169 non-null  float64
 9   Motion/Activity index  12169 non-null  float64
 10  PVI                    12169 non-null  float64
 11  Hb level               12169 non-null  float64
 12  SV                     12169 non-null  float64
 13  CO                     12169 non-null  float64
 14  Blood Flow Index       12169 non-null  float64
 15  PP

None

Dataset: patient.csv
Počet riadkov:  2189
Počet stĺpcov:  13


...

None

Dataset: station.csv
Počet riadkov:  711
Počet stĺpcov:  6


,QoS,station,latitude,revision,longitude,location
0,maintenance,Baki,9.89206,2024-10-22,43.38531,Africa/Mogadishu
1,good,Voznesensk,47.56494,"11/20/2023, 00:00:00",31.33078,Europe/Kyiv
2,good,Msaken,35.72917,21 Apr 2017,10.58082,Africa/Tunis
3,acceptable,Pottstown,40.24537,13 Nov 2019,-75.64963,America/New_York
4,acceptable,Gilbert,33.35283,2023-09-04,-111.78903,America/Phoenix


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   QoS        711 non-null    object 
 1   station    711 non-null    object 
 2   latitude   711 non-null    float64
 3   revision   711 non-null    object 
 4   longitude  711 non-null    float64
 5   location   711 non-null    object 
dtypes: float64(2), object(4)
memory usage: 33.5+ KB


None

### 1.1(A) Analýza štruktúr dát
Projekt obsahuje 3 základné súbory:

- **observation.csv** – obsahuje senzorové merania pacienta (napr. SpO₂, HR, RR, tlak, teplotu kože,a iné)
- **patient.csv** – uchováva informácie o pacientoch ako základné(meno, priezvisko,mail,...) tak aj pokročilé(krvná skupina,dátum narodenia,...)
  

- **station.csv** – obsahuje informácie o staniciach, kde boli merania vykonávané (napr.location, stationm, QoS).


Z hľadiska dátových typov sú všetky súbory v poriadku – numerické premenné majú typ `float64`
a textové typ `object`. Dátumové a časové údaje možno budú vyžadovať
konverziu na typ `datetime64` 




In [23]:
print(df_obs.columns.tolist())
print(df_pat.columns.tolist())
print(df_sta.columns.tolist())

['SpO₂', 'HR', 'PI', 'RR', 'EtCO₂', 'FiO₂', 'PRV', 'BP', 'Skin Temperature', 'Motion/Activity index', 'PVI', 'Hb level', 'SV', 'CO', 'Blood Flow Index', 'PPG waveform features', 'Signal Quality Index', 'Respiratory effort', 'O₂ extraction ratio', 'SNR', 'oximetry', 'latitude', 'longitude']
['residence', 'name', 'username', 'blood_group', 'ssn', 'mail', 'registration', 'job', 'user_id', 'address', 'company', 'birthdate', 'station_ID']
['QoS', 'station', 'latitude', 'revision', 'longitude', 'location']
